### Scraping: HTTP Request, HTML Parsing

In [ ]:
import requests
from bs4 import BeautifulSoup

url = "https://idexuae.ae/exhibit/exhibitor-list"
response = requests.get(url)

print(response.status_code)
print(response.content)

soup = BeautifulSoup(response.content, "html5lib")

print(soup.content)

divs = soup.find_all("div", class_= "row")
for div in divs:
    print(div.get_text())

### Web API: HTTP Request, JSON Parsing

In [ ]:
import requests
import pandas as pd
import sqlite3
from tqdm import tqdm


# 1 dataframe
df = pd.DataFrame(columns=["name", "website", "country", "description"])

# 2 requests
base_url = "https://ezone.idexuae.ae/exhibitors-list-json/5?event_id=5&orderby=name&tag=_all_&search=_all_&start=_all_&page="
headers = {'Accept': 'application/json'}

for i in tqdm(range(0, 95)):
    page_url = base_url + str(i + 1)
    response_json = requests.get(page_url, headers=headers)
    response_deserialised = response_json.json()
    for item in response_deserialised["data"]:
        new_row = pd.DataFrame({
            "name": item["name"], 
            "website": item["website"], 
            "country": item["country"]["name"], 
            "description": item["description"]
            }, index=[0])
        df = pd.concat([new_row, df])

# 3 database
connection = None
try:
    connection = sqlite3.connect("results/idex2023exhibitors.db")
except sqlite3.Error as e:
    print(e)

df.to_sql("exhibitors", connection, if_exists="replace", index=False)

connection.commit()
